In [1]:
source("aux.R")

Loading required package: NLP

Loading required package: koRpus.lang.en

Loading required package: koRpus

Loading required package: sylly

For information on available language packages for 'koRpus', run

  available.koRpus.lang()

and see ?install.koRpus.lang()



Attaching package: ‘koRpus’


The following object is masked from ‘package:tm’:

    readTagged



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
dataset <- read.csv("train.csv", col.names = c("Label", "Text", "Tag"))
classes <- as.integer(sort(unique(dataset$Label)))
args <- sort(unique(unlist(strsplit(dataset$Tag, ","))))

head(dataset)
cat(classes, "\n")
cat(args)

,Label,Text,Tag
,<int>,<chr>,<chr>
1,1,Says the Annies List political group supports third-trimester abortions on demand.,abortion
2,2,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments"
3,3,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy
4,1,Health care reform legislation is likely to mandate free sex change surgeries.,health-care
5,2,The economic turnaround started at the end of my term.,"economy,jobs"
6,5,The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades.,education


0 1 2 3 4 5 
10-news-tampa-bay abc-news-week abortion afghanistan after-the-fact agriculture Alcohol animals autism bankruptcy baseball bipartisanship bush-administration campaign-advertising campaign-finance candidates-biography cap-and-trade census children china city-budget city-government civil-rights climate-change colbert-report congress congressional-rules consumer-safety corporations corrections-and-updates county-budget county-government crime criminal-justice death-penalty debates debt deficit disability diversity drugs ebola economy education elections energy environment ethics fake-news families federal-budget financial-regulation fires florida florida-amendments food food-safety foreign-policy gambling gas-prices gays-and-lesbians government-efficiency government-regulation guns health-care history homeland-security homeless housing human-rights hunger immigration income infrastructure iraq islam israel job-accomplishments jobs kagan-nomination labor legal-issues lottery m

In [3]:
dataset$Text <- clean(dataset$Text)
dataset <- clean_empty_rows(dataset)
head(dataset)

,Label,Text,Tag
,<int>,<chr>,<chr>
1,1,say anni list polit group support thirdtrimest abort demand,abortion
2,2,declin coal start start natur gas take start begin presid georg w bush administr,"energy,history,job-accomplishments"
3,3,hillari clinton agre john mccain vote give georg bush benefit doubt iran,foreign-policy
4,1,health care reform legisl like mandat free sex chang surgeri,health-care
5,2,econom turnaround start end term,"economy,jobs"
6,5,chicago bear start quarterback last year total numb tenur uw faculti fire last two decad,education


In [4]:
vocabulary <- get_vocabulary(dataset$Text)
cat(length(vocabulary))

2255

In [28]:
new_dataset <- dataset[1:5000, ]

In [29]:
model <- train_multinomial_nb(classes, new_dataset)
#print(model$vocab)
#print(model$prior)
#print(model$post)

In [34]:
print(model$post)

             2            3            4       abbott         abil          abl
0 0.0005795086 0.0001159017 1.159017e-04 0.0003477051 0.0003477051 0.0005795086
1 0.0009006304 0.0005003502 4.002802e-04 0.0002001401 0.0004002802 0.0008005604
2 0.0008021390 0.0007130125 8.912656e-05 0.0004456328 0.0002673797 0.0004456328
3 0.0010467219 0.0005709392 2.854696e-04 0.0002854696 0.0003806261 0.0008564088
4 0.0002015723 0.0004031445 2.015723e-04 0.0004031445 0.0002015723 0.0002015723
5 0.0005660591 0.0005660591 2.264236e-04 0.0001132118 0.0001132118 0.0002264236
       abolish        abort      absolut         abus       accept       access
0 1.159017e-04 0.0018544274 0.0003477051 2.318034e-04 0.0004636069 0.0002318034
1 6.004203e-04 0.0031021715 0.0001000700 4.002802e-04 0.0005003502 0.0007004903
2 2.673797e-04 0.0014260250 0.0001782531 2.673797e-04 0.0002673797 0.0004456328
3 9.515653e-05 0.0008564088 0.0001903131 9.515653e-05 0.0003806261 0.0003806261
4 4.031445e-04 0.0012094336 0.0004031445

In [31]:
apply_multinomial_nb <- function(classes, vocab, prior, condprob, doc) {
  # Extract tokens from the document that are in the vocabulary
  tokens <- intersect(unlist(strsplit(doc, "\\s+")), vocab)
  
  # Initialize scores for each class
  scores <- numeric(length(classes))
  
  for (i in seq_along(classes)) {
    # Initialize score for each class with log of prior probability
    score <- log(prior[i])
    
    for (term in tokens) {
      # Check if term exists in the vocabulary and has conditional probabilities
      if (term %in% condprob && !is.na(condprob[[term]][i])) {
        score <- score + log(condprob[[term]][i])
      }
    }
    
    # Store the score for the current class
    scores[i] <- score
  }
  
  # Return class with the highest score
  return(classes[which.max(scores)])
}

In [32]:
# Apply the model to a new document
test_docs <- dataset[5000:6000, ]
head(test_docs)

,Label,Text,Tag
,<int>,<chr>,<chr>
5000,1,say daniel webster think wife submit husband,"candidates-biography,marriage,message-machine"
5001,0,say fight repeal septic tank inspect program tallahasse pull fast one,"environment,message-machine-2012"
5002,0,onehundr time mani peopl work way poverti american middl class presid compar ronald reagan presid,"economy,job-accomplishments,jobs,poverty,workers"
5003,4,new hartsfieldjackson intern termin time budget,transportation
5004,0,scott walker give tax deduct millionair send kid exclus privat school,"education,income,taxes,wealth"
5005,1,sheldon whitehous senat vote censor internet,criminal-justice


In [33]:
docs <- test_docs$Text

pred_labels <- sapply(docs, function(doc) {
  apply_multinomial_nb(classes, model$vocab, model$prior, model$post, doc)
})

print(pred_labels)

                                                                                                                                                          say daniel webster think wife submit husband 
                                                                                                                                                                                                     1 
                                                                                                                                 say fight repeal septic tank inspect program tallahasse pull fast one 
                                                                                                                                                                                                     1 
                                                                                                     onehundr time mani peopl work way poverti american middl class presid compar ronald reagan presid 


In [22]:
correct_predictions <- sum(test_docs$Label == pred_labels)
total_predictions <- length(test_docs$Label)
accuracy <- correct_predictions / total_predictions

cat("Accuracy:", accuracy)

Accuracy: 0.2077922